In [56]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

In [53]:
DIRECTORY = r'./data'
CATEGORIES = ['vehicles', 'non-vehicles']
IMG_SIZE = 64

X = []
y = []

def create_train_data():
    for category in CATEGORIES:
        path = os.path.join(DIRECTORY, category)
        label = 1 if category == 'vehicles' else 0

        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                X.append(new_array)
                y.append(label)
            except Exception as e:
                pass

create_train_data()

In [54]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [55]:
import pickle

pickle_out = open('X_train.pickle', 'wb')
pickle.dump(X_train, pickle_out)
pickle_out.close()

pickle_out = open('y_train.pickle', 'wb')
pickle.dump(y_train, pickle_out)
pickle_out.close()

In [61]:
X = pickle.load(open('X_train.pickle', 'rb'))
y = pickle.load(open('y_train.pickle', 'rb'))
X = X/255.0    # Normalizes data

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=64, epochs=9, validation_split=0.2)

Train on 11366 samples, validate on 2842 samples
Epoch 1/9
11366/11366 [==============================] - 48s 4ms/sample - loss: 0.2703 - acc: 0.8797 - val_loss: 0.1146 - val_acc: 0.9613
Epoch 2/9
11366/11366 [==============================] - 46s 4ms/sample - loss: 0.1034 - acc: 0.9610 - val_loss: 0.0800 - val_acc: 0.9726
Epoch 3/9
11366/11366 [==============================] - 46s 4ms/sample - loss: 0.0739 - acc: 0.9744 - val_loss: 0.0675 - val_acc: 0.9778
Epoch 4/9
11366/11366 [==============================] - 46s 4ms/sample - loss: 0.0603 - acc: 0.9775 - val_loss: 0.0690 - val_acc: 0.9799
Epoch 5/9
11366/11366 [==============================] - 46s 4ms/sample - loss: 0.0449 - acc: 0.9835 - val_loss: 0.0637 - val_acc: 0.9792
Epoch 6/9
11366/11366 [==============================] - 46s 4ms/sample - loss: 0.0396 - acc: 0.9868 - val_loss: 0.1258 - val_acc: 0.9624
Epoch 7/9
11366/11366 [==============================] - 46s 4ms/sample - loss: 0.0336 - acc: 0.9871 - val_loss: 0.0546 - v

In [62]:
model.save('model.model')